In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import LinearSVC

#from keras.models import Sequential
#from keras.layers import Dense
#from keras import regularizers
#from keras import optimizers

#from keras.layers import Dropout
#from keras.utils import to_categorical
#import tensorflow as tf

In [2]:
#cites = pd.read_csv('./data/Train/2012-13/2012-13_cites.csv')
#judges = pd.read_csv('./data/Train/2012-16X1_judges.csv')
target = pd.read_csv('./data/Train/2012-13/2012-13_targetscore.csv')
#X_test = pd.read_csv('./data/test/2014/2014_cites.csv')
#y_test = pd.read_csv('./data/test/2014/2014X1_targetscore.csv')

In [3]:
len(cites)

3935

In [ ]:
#len(judges)

In [3]:
len(target)

3935

In [4]:
target['target_score'].value_counts()

-2    2291
-1     788
 0     283
 2     268
 1     111
 3      81
-3      80
 4      26
 5       4
-4       3
Name: target_score, dtype: int64

In [7]:
0.6695501730103807-(2291/3935.0)

0.08733924543731841

In [ ]:
target['flat_target'] = target['target_score'].apply(lambda x: 0 if x < 0 else 1)

In [ ]:
target['flat_target'].value_counts()

In [ ]:
1573/1925.0

In [ ]:
big_df = cites.merge(judges, left_index=True, right_index=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cites, target['target_score'], test_size=.33)

In [ ]:
# Prepare classification models
models = []
models.append(('LOG', LogisticRegression()))
#models.append(('SGDC', SGDClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('RFRST', RandomForestClassifier()))
models.append(('LSVM', LinearSVC()))
models.append(('MNB', MultinomialNB()))
# Evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = KFold(n_splits=3)
    cv_results = cross_val_score(model, cites, target['target_score'], cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
target['target_score'].value_counts()

In [ ]:
1082/1925.0

In [ ]:
log = LogisticRegression()

log_param = {'C':np.logspace(-4, 4, 10000), 'solver':['newton-cg', 'lbfgs', 'sag', 'saga'], 'class_weight':['balanced', None], 'max_iter':np.linspace(100, 1000, 10000)}

log_ran = RandomizedSearchCV(log, log_param, n_iter=5, cv=3, verbose=2, n_jobs=1)
log_ran.fit(cites, target['target_score'])
print log_ran.best_estimator_.score(X_test, y_test)
print log_ran.best_params_

In [5]:
rf = RandomForestClassifier()

rf_param = {'n_estimators': np.random.randint(10, 1000, 10000), 'criterion':['gini', 'entropy'], 'class_weight':[None, 'balanced']}

rf_ran = RandomizedSearchCV(rf, rf_param, n_iter=5, cv=3, verbose=2, n_jobs=1)
rf_ran.fit(cites, target['target_score'])
print rf_ran.best_estimator_.score(X_test, y_test)
print rf_ran.best_params_

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=76, criterion=gini, class_weight=balanced ..........
[CV]  n_estimators=76, criterion=gini, class_weight=balanced, total=  20.2s
[CV] n_estimators=76, criterion=gini, class_weight=balanced ..........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.8s remaining:    0.0s


[CV]  n_estimators=76, criterion=gini, class_weight=balanced, total=  19.2s
[CV] n_estimators=76, criterion=gini, class_weight=balanced ..........
[CV]  n_estimators=76, criterion=gini, class_weight=balanced, total=  19.3s
[CV] n_estimators=323, criterion=entropy, class_weight=balanced ......
[CV]  n_estimators=323, criterion=entropy, class_weight=balanced, total= 1.4min
[CV] n_estimators=323, criterion=entropy, class_weight=balanced ......
[CV]  n_estimators=323, criterion=entropy, class_weight=balanced, total= 1.3min
[CV] n_estimators=323, criterion=entropy, class_weight=balanced ......
[CV]  n_estimators=323, criterion=entropy, class_weight=balanced, total= 1.3min
[CV] n_estimators=78, criterion=gini, class_weight=None ..............
[CV]  n_estimators=78, criterion=gini, class_weight=None, total=  20.8s
[CV] n_estimators=78, criterion=gini, class_weight=None ..............
[CV]  n_estimators=78, criterion=gini, class_weight=None, total=  20.7s
[CV] n_estimators=78, criterion=gini, 

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 27.0min finished


0.6695501730103807
{'n_estimators': 999, 'criterion': 'entropy', 'class_weight': 'balanced'}


In [ ]:
conmat_flat = np.array(confusion_matrix(y_test, log_ran.predict(X_test)))

confusion_flat = pd.DataFrame(conmat_flat, index=['is_-1', 'is_0', 'is_1'],
                         columns=['pre_-1', 'pre_0', 'pre_1'])
confusion_flat

In [7]:
conmat = np.array(confusion_matrix(y_test, rf_ran.best_estimator_.predict(X_test)))

confusion = pd.DataFrame(conmat, index=['is_-4', 'is_-3', 'is_-2','is_-1', 'is_0', 'is_1', 'is_2', 'is_3', 'is_4', 'is_5'],
                         columns=['pre_-4','pre_-3', 'pre_-2', 'pre_-1', 'pre_0', 'pre_1', 'pre_2', 'pre_3', 'pre_4', 'pre_5'])
confusion

,pre_-4,pre_-3,pre_-2,pre_-1,pre_0,pre_1,pre_2,pre_3,pre_4,pre_5
is_-4,0,0,2,0,0,0,0,0,0,0
is_-3,0,0,23,1,1,0,0,0,0,0
is_-2,0,0,900,52,0,2,0,0,0,0
is_-1,0,0,86,254,1,2,0,0,0,0
is_0,0,0,115,21,0,1,1,0,1,0
is_1,0,0,37,11,0,4,0,0,0,0
is_2,0,0,119,15,0,0,3,0,1,0
is_3,0,0,52,14,0,0,0,0,0,0
is_4,0,0,8,3,0,1,0,0,0,0
is_5,0,0,3,0,0,0,0,0,0,0


In [8]:
print classification_report(y_test, rf_ran.best_estimator_.predict(X_test))

             precision    recall  f1-score   support

         -4       0.00      0.00      0.00         2
         -3       0.00      0.00      0.00        25
         -2       0.67      0.94      0.78       954
         -1       0.68      0.74      0.71       343
          0       0.00      0.00      0.00       139
          1       0.40      0.08      0.13        52
          2       0.75      0.02      0.04       138
          3       0.00      0.00      0.00        66
          4       0.00      0.00      0.00        12
          5       0.00      0.00      0.00         3

avg / total       0.58      0.67      0.58      1734



/home/loki/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
model = Sequential()
model.add(Dense(judges.shape[1], input_dim=judges.shape[1], activation='relu'))
model.add(Dense(judges.shape[1], activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(9, activation='softmax'))

judge_y = to_categorical(target['target_score'], num_classes=9)
adam = optimizers.Adam(lr=.001)

In [ ]:
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
target['target_score'].value_counts()

In [ ]:
fit_judge_nn = model.fit(judges, judge_y, epochs=150)